In [1]:
#导入模块
import requests
import pandas as pd
import numpy as np
import json
import sklearn
import matplotlib.pyplot as plt
import seaborn as sn
import random
from sklearn.cluster import DBSCAN
import folium
from folium import plugins
from folium import FeatureGroup, Marker, LayerControl
import os

In [13]:
path = r"C:\Users\wangchuan\data"
dirs = os.listdir(path)
for file in os.listdir(path):
    file_path = os.path.join(path, file)
    print(file_path)

C:\Users\wangchuan\data\.gitkeep
C:\Users\wangchuan\data\acs_ny.csv
C:\Users\wangchuan\data\banklist.csv
C:\Users\wangchuan\data\billboard.csv
C:\Users\wangchuan\data\bladder.csv
C:\Users\wangchuan\data\concat_1.csv
C:\Users\wangchuan\data\concat_2.csv
C:\Users\wangchuan\data\concat_3.csv
C:\Users\wangchuan\data\country_timeseries.csv
C:\Users\wangchuan\data\doctors.csv
C:\Users\wangchuan\data\doctors_unicode.csv
C:\Users\wangchuan\data\epi_sim.txt
C:\Users\wangchuan\data\epi_sim.zip
C:\Users\wangchuan\data\gapminder.tsv
C:\Users\wangchuan\data\housing.csv
C:\Users\wangchuan\data\housing_renamed.csv
C:\Users\wangchuan\data\pew.csv
C:\Users\wangchuan\data\raw_data_urls.txt
C:\Users\wangchuan\data\scientists.csv
C:\Users\wangchuan\data\survey_person.csv
C:\Users\wangchuan\data\survey_site.csv
C:\Users\wangchuan\data\survey_survey.csv
C:\Users\wangchuan\data\survey_visited.csv
C:\Users\wangchuan\data\tesla_stock_yahoo.csv
C:\Users\wangchuan\data\weather.csv
C:\Users\wangchuan\data\wine.cs

numpy.matrix

In [2]:
# 参数模块

#文件名,后续有可能做成列表
work_dir = './data/'
# name = '桐庐.xlsx'
names = os.listdir(work_dir)
out_dir = './html/'



hotel_adress = '汉庭酒店(杭州临安万华店)'
hotel_city= '杭州市'
# hotel_point = [0,0]

# 聚类参数
eps=0.03
min_samples=3

#ga参数
CROSS_RATE = 0.1
MUTATE_RATE = 0.1
POP_SIZE = 500
N_GENERATIONS = 500



In [9]:
#main
for name in names: #便利文件夹中每个文件
    hotel_point = get_hotel_point(hotel_adress,hotel_city)
    df = get_data(work_dir,name)
    groups_dict = get_groups_dict(df)
    html = My_html(hotel_point) #初始化画图对象
    colors = ['red','green','blue','yellow']

    #主循环
    for i in range(3):
        dis_dict = get_min_close(groups_dict) #获取距离字典
        closest_label = sorted(get_min_close(groups_dict).items(), key=lambda obj: obj[1])[0][0] #选择最近的label
        # random_label = random.choice(list(groups_dict.keys())) #随机取出一个组
        uesd_df = get_group_and_upgrade_groups_dict(closest_label) #这些点直接开始塞

        #GA初始化
        N_CITIES = len(uesd_df)# DNA size
        ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)
        city_position = uesd_df[['经度','纬度']].values
        #GA训练
        for generation in range(N_GENERATIONS):
            lx, ly = ga.translateDNA(ga.pop, city_position)
            fitness, total_distance = ga.get_fitness(lx, ly)
            ga.evolve(fitness)
            best_idx = np.argmax(fitness)
            # print('Gen:', generation, '| best fit: %.2f' % total_distance[best_idx],)
        #GA成果导出，result_df 只是按照ga的结果重新排序的used_df
        # result_df = uesd_df.iloc[ga.pop[best_idx],:]

        result_df = uesd_df.copy()
        result_df = result_df.set_index('经度')
        result_df = result_df.loc[lx[best_idx],:].drop_duplicates('公司名称').reset_index()

        #开始画图
        html.plot_point_and_line(result_df['纬度'],result_df['经度'],result_df['公司名称'],color=colors[i])
    html.plot_point_and_line([hotel_point[1]],[hotel_point[0]],['旅馆'],color='black',icon=folium.Icon(color='black', icon='home',))
    html_name = name.split('.')[0]+'.html' #保存的图片名称
    html.save(out_dir+html_name) #保存图片

ValueError: check_hostname requires server_hostname

In [3]:
def get_hotel_point(hotel_adress,hotel_city):
    '''
    获取旅馆经纬度
    '''
    key = '676288229ff9fa23f7bcebf6b1dc510a'
    url = r'https://restapi.amap.com/v3/place/text?key=%s&keywords=%s&citylimit=%s'%(key,hotel_adress,hotel_city)

    result = requests.get(url)
    a = json.loads(result.text)
    tmp = a['pois']
    if len(tmp)>0:

        hotel_point = list(map(float,a['pois'][0]['location'].split(',')))
    return hotel_point

In [4]:
def get_data(work_dir,name):
    #读取临安组的最新数据
    df = pd.read_excel(work_dir+name,index_col='index')  #这里从腾讯文档下载数据，要将数据打开查看，将多余的列处理，不然后面dropna会带来很大麻烦
    df = df[df['是否调研']!=1] #去掉已经调研过得点
    df =df[['公司名称','经度','纬度','分组']] #留下有用的字段
    df.dropna(inplace=True)
    return df

In [5]:
def get_groups_dict(df):
    '''
    使用密度聚类，获得label。返回一个label为建，group_df为值得dict
    '''
    X = df[['经度','纬度']].astype('float').values #X是传入聚类算法的预测数据
    clf = DBSCAN(eps = eps, min_samples = min_samples) #初始化聚类算法
    y_pred = clf.fit_predict(X) #使用密度聚类进行预测。y_pred是一维向量，返回了各个数据对应的组别
    df['label'] = y_pred #将得到的预测结果保存到初始的df中
    df_clean = df[(df['label'] != -1)] #干掉孤立点
    df_clean['distance'] = np.sqrt(np.square(df_clean['经度'] - hotel_point[0]) +np.square(df_clean['纬度'] - hotel_point[1])) #计算各点到旅馆的距离
    groups = df_clean.groupby('label') #按照label对df_clean进行分组。df_clean可以理解为df.copy()
    groups_dict = {} #因为要对group进行删改，构建一个group_dict存放df
    for label,group in groups:
        if len(group)>5: #剔除长度小于5的类别
            groups_dict[label] = group
    return groups_dict

In [6]:
def get_min_close(dict_):
    '''
    制作各类与group的最近距离
    '''
    dis_dict = {}
    for label,group in dict_.items():
        dis_dict[label] = group['distance'].min()
    return dis_dict
def get_num(df):
    '''
    get_group_and_upgrade_groups_dict的辅助函数
    根据df剩余的长度，决定是全部取完，还是先取10个，如果全部取完，则flag返回1
    '''
    length = len(df)
    flag = 0 #这个组是否被取完
    if length <= 20:
        number = length
        flag=1
    else:
        number = 10
    return  flag, number

def get_top_10_df(df):
    '''
    get_group_and_upgrade_groups_dict的辅助函数
    如果一个df不取完，只取10个，那么先找离宾馆最近的点A，再找离A点最近的10个点（包括A本身）
    '''
    df.sort_values('distance',inplace=True)
    a_index = df.index[0]
    jingdu,weidu = df.loc[a_index,['经度','纬度']].values
    df['distance2'] = ((df['经度']-jingdu)**2+(df['纬度']-weidu)**2)**0.5
    df.sort_values('distance2',inplace=True)
    return df.head(10)

def get_group_and_upgrade_groups_dict(random_label):
    '''
    获取要进行tsp规划的df，同时更新groups_dict
    '''
    flag, number = get_num(groups_dict[random_label])
    if flag==1:
        group = groups_dict[random_label] 
        del groups_dict[random_label] 
        return group
    elif flag==0:
        top10_df = get_top_10_df(groups_dict[random_label])
        groups_dict[random_label] = groups_dict[random_label][~groups_dict[random_label].index.isin(top10_df.index)]
        return top10_df

In [7]:
#TSP辅助函数与类


class GA(object):
    #遗传算法主类
    def __init__(self, DNA_size, cross_rate, mutation_rate, pop_size, ):
        self.DNA_size = DNA_size
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size
        self.pop = np.vstack([np.random.permutation(DNA_size) for _ in range(pop_size)])

    def translateDNA(self, DNA, city_position):     # get cities' coord in order
        line_x = np.empty_like(DNA, dtype=np.float64)
        line_y = np.empty_like(DNA, dtype=np.float64)
        for i, d in enumerate(DNA):
            city_coord = city_position[d]
            line_x[i, :] = city_coord[:, 0]
            line_y[i, :] = city_coord[:, 1]
        return line_x, line_y

    def get_fitness(self, line_x, line_y):
        total_distance = np.empty((line_x.shape[0],), dtype=np.float64)
        for i, (xs, ys) in enumerate(zip(line_x, line_y)):
            # total_distance[i] = np.sum(np.sqrt(np.square(np.append(np.diff(xs),xs[-1]-xs[0])) + np.square(np.append(np.diff(ys),ys[-1]-ys[0]))))
            total_distance[i] = np.sum(np.sqrt(np.square(np.diff(xs)) + np.square(np.diff(ys))))
        total_distance*=10
        fitness = np.exp(self.DNA_size * 2 / total_distance)
        self.total_distance = total_distance
        return fitness, total_distance

    def select(self, fitness):
        try:
            idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=(fitness) / (fitness).sum())
        except:
            print((fitness))
        return self.pop[idx]

    def crossover(self, parent, pop):
        if np.random.rand() < self.cross_rate:
            i_ = np.random.randint(0, self.pop_size, size=1)                        # select another individual from pop
            cross_points = np.random.randint(0, 2, self.DNA_size).astype(np.bool)   # choose crossover points
            keep_city = parent[~cross_points]                                       # find the city number
            swap_city = pop[i_, np.isin(pop[i_].ravel(), keep_city, invert=True)]
            parent[:] = np.concatenate((keep_city, swap_city))
        return parent

    def mutate(self, child):
        for point in range(self.DNA_size):
            if np.random.rand() < self.mutate_rate:
                swap_point = np.random.randint(0, self.DNA_size)
                swapA, swapB = child[point], child[swap_point]
                child[point], child[swap_point] = swapB, swapA
        return child

    def evolve(self, fitness):
        pop = self.select(fitness)
        pop_copy = pop.copy()
        for i,parent in enumerate(pop_copy):  # for every parent
            child = self.crossover(parent, pop_copy)
            child = self.mutate(child)
            pop_copy[i,:] = child
        self.pop = pop_copy


In [8]:
#画图模块
class My_html():
    def __init__(self,location):
        self.san_map = folium.Map(
                            location=[location[1],location[0]], #杭州为中心，不改了
                            zoom_start=16,
                            # 高德街道图
                            tiles='http://webrd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&scale=1&style=7&x={x}&y={y}&z={z}',
                            attr='default')
        self.marker_cluster = plugins.MarkerCluster().add_to(self.san_map)

    def plot_point_and_line(self,X_lat,X_lon,address_names,color,**kargs): 
        '''
        天坑，要先传入纬度，后传入经度，神经病
        '''

        folium.PolyLine(np.array(list(zip(X_lat, X_lon))), color=color).add_to(self.san_map)
        for x, y, name in zip(X_lat,X_lon,address_names):
            folium.Marker([x, y], color='balck',popup=name,**kargs).add_to(self.marker_cluster)

    def save(self,name):
        self.san_map.save(name)

In [ ]:
sparse coding 意见领袖 sna